In [97]:
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import json

import unicodedata
import re

from pydantic import BaseModel, Field

In [98]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

In [99]:
class Translation(BaseModel):
    """The updated translation, and the comments for the update"""
    updated_translation: str = Field(description="The updated translation")
    comment: str = Field(description="Comments on the translation update")

In [100]:
system_instruction_prompt = """You are a professional translator. You know Japanese, English and Chinese. You can translate Japanese into either Chinese or English."""

client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
    max_output_tokens = 8192,
    response_mime_type = 'application/json',
    response_schema = Translation,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=system_instruction_prompt)],
  )

In [101]:
reflection_prompt = """You are translating the following Japanese text into Chinese. You have an initial Chinese translation and an initial English translation.

Criticise the Chinese translation based on the Japanese text. Compare the Chinese translation with the English translation.

Improve the Chinese translation. Keep the originial Japanese meanings. Use a more conversational style. Display in Traditional Chinese.

This is the Japanese text.
{japanese_text}

This is the initial Chinese translation.
{chinese_translation}

This is the English translation.
{english_translation}
"""


In [102]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output




In [103]:
with open(f"raw_translations_main_text_json/Chiri_Yukie_prologue.json", "r", encoding="utf8") as f:
    data_dict = json.load(f)

In [104]:
# Read the markdown template for writing the Chinese translations to Markdown file

#read in the template
with open("templates/iter1_output_md_template", "r", encoding="utf8") as f:
    md_template = f.read()
    md_template = unicodedata.normalize('NFKC', md_template)

In [105]:
reflection_prompt_formatted = reflection_prompt.format(japanese_text=data_dict['japanese_translation'],chinese_translation=data_dict['chinese_translation'],english_translation=data_dict['english_translation'])

In [106]:
updated_translation = generate(client,generate_content_config,model,prompt = reflection_prompt_formatted)

{
  "comment": "The initial Chinese translation is generally accurate but a bit too formal and literal. It lacks the emotional depth and conversational tone present in the Japanese original. The English translation captures the essence and emotional nuances better. The revised Chinese translation aims to address these issues by using more natural and evocative language, and by incorporating elements that reflect the Ainu culture's connection to nature and their sense of loss.",
  "updated_translation": "序\n\n很久很久以前，這片廣闊的北海道，是我們祖先自由自在的天地。他們就像天真爛漫的孩子，被美麗的大自然擁抱著，無憂無慮地生活，真是大自然的寵兒，多麼令人稱羨的生活啊！\n\n在寒冷的冬天，他們踢開覆蓋森林的厚厚積雪，不畏懼那冰凍天地的寒氣，翻山越嶺地去獵熊；在涼風徐徐的夏天，他們在綠波蕩漾的海面上，與海鷗的歌聲為伴，划著像樹葉般的小船，整天辛勤地捕魚；在百花盛開的春天，他們沐浴在溫暖的陽光下，與不停歌唱的小鳥一同嬉戲，採集蕗和蓬；在紅葉似火的秋天，他們撥開被秋風吹拂、穗頭飽滿的芒草，直到深夜捕鮭魚的篝火都已熄滅，在山谷間迴盪的鹿鳴聲中，對著皎潔的月亮編織著美麗的夢想。啊，那是多麼快樂的生活啊！那和平寧靜的時光，如今都已成為過往雲煙。夢想破滅至今已數十年，這片土地經歷了快速的變遷，山野變成了村莊，村莊又變成了城鎮，一步步地被開墾發展。\n\n太古時代那原始的自然風貌，不知在何時已經悄然褪色，曾經在田野山林間快樂生活的人們，如今又都去了哪裡呢？僅存的我們這些族人，對於這不斷變化的世界，只能睜大眼睛，感到驚訝和茫然。而且，從我們的眼神中，已經失去了昔日祖先那種被宗教情懷

In [107]:
updated_translation_dict = json.loads(updated_translation)
print(updated_translation_dict['updated_translation'])
print(updated_translation_dict['comment'])

序

很久很久以前，這片廣闊的北海道，是我們祖先自由自在的天地。他們就像天真爛漫的孩子，被美麗的大自然擁抱著，無憂無慮地生活，真是大自然的寵兒，多麼令人稱羨的生活啊！

在寒冷的冬天，他們踢開覆蓋森林的厚厚積雪，不畏懼那冰凍天地的寒氣，翻山越嶺地去獵熊；在涼風徐徐的夏天，他們在綠波蕩漾的海面上，與海鷗的歌聲為伴，划著像樹葉般的小船，整天辛勤地捕魚；在百花盛開的春天，他們沐浴在溫暖的陽光下，與不停歌唱的小鳥一同嬉戲，採集蕗和蓬；在紅葉似火的秋天，他們撥開被秋風吹拂、穗頭飽滿的芒草，直到深夜捕鮭魚的篝火都已熄滅，在山谷間迴盪的鹿鳴聲中，對著皎潔的月亮編織著美麗的夢想。啊，那是多麼快樂的生活啊！那和平寧靜的時光，如今都已成為過往雲煙。夢想破滅至今已數十年，這片土地經歷了快速的變遷，山野變成了村莊，村莊又變成了城鎮，一步步地被開墾發展。

太古時代那原始的自然風貌，不知在何時已經悄然褪色，曾經在田野山林間快樂生活的人們，如今又都去了哪裡呢？僅存的我們這些族人，對於這不斷變化的世界，只能睜大眼睛，感到驚訝和茫然。而且，從我們的眼神中，已經失去了昔日祖先那種被宗教情懷所支配的美麗靈魂的光輝，取而代之的是不安和不滿，變得遲鈍而黯淡，看不清未來的方向，不得不依靠別人的憐憫，這是多麼可悲的模樣啊！唉，這正在消逝的事物……這就是我們現在的名字，我們背負著多麼悲傷的名字啊！

很久以前，我們那些幸福的祖先，一定連想都沒想到，自己深愛的這片故土，最終會變成如此淒涼的景象吧。

時間不停地流逝，世界不斷地發展。在這激烈的競爭場中，即使現在的我們正暴露著失敗的醜態，但總有一天，如果能出現兩三位堅強的人，我們也能夠跟上時代的步伐，與世界並肩前行吧。這真的是我們最迫切的願望，也是我們日夜不停祈禱的事情。

但是……我們敬愛的祖先，為了在日常生活中彼此溝通而使用的那些古老的語言，那些被傳承下來的美麗詞語，難道也要隨著我們這些弱者一同消失殆盡嗎？啊，這實在是太令人悲痛和惋惜了！

我身為阿伊努人，在阿伊努語的環境中長大，每當在下雨的夜晚，下雪的夜晚，只要有空閒，就會聚集在一起，用我笨拙的筆，記錄下我們的祖先所講述的無數故事中，極小的一部分。

如果能讓更多了解我們的人讀到這些文字，我將與我的阿伊努同胞和祖先們一同，感受到無比的喜悅和至上的幸福。

大正十一年三月一日

知里幸惠
The initial

In [108]:
data_dict['updated_chinese_translation'] = updated_translation_dict['updated_translation']
data_dict['update_comment'] = updated_translation_dict['comment']

md_output = md_template.format(translated_language="Chinese", ainu_title=data_dict['ainu_title'],
                formatted_prompt=reflection_prompt_formatted,
                japanese_title=data_dict['japanese_title'], input_japanese = data_dict['japanese_translation'],
                output= f"{data_dict['updated_chinese_translation']}\n\n{data_dict['update_comment']}")

with open(f"updated_translations_main_text_iter1/Chinese_Translation_JSON/Chiri_Yukie_prologue.json", "w", encoding="utf8") as f:
    json.dump(data_dict, f, ensure_ascii=False, indent=4)

with open(f"updated_translations_main_text_iter1/Chinese_Translation/Chiri_Yukie_prologue.md", "w", encoding="utf8") as f:
    f.write(md_output)


